In [2]:
import os, shutil, glob
import time, requests
import datetime as datetime
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
os.makedirs(os.path.join("Cellphones Data","Dataset"),exist_ok = True)

In [4]:
def Scrape_Product(URLs, Name):
    driver = webdriver.Edge()

    try:
        URL = URLs
        driver.get(URL)
        
        def close_popups():
            while True:
                try:
                    close_button = WebDriverWait(driver, 2).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, "button.modal-close.is-large"))
                    )
                    close_button.click()
                    time.sleep(0.5)
                except Exception:
                    break

            while True:
                try:
                    close_promo_button = WebDriverWait(driver, 2).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, "button.cancel-button-top"))
                    )
                    close_promo_button.click()
                    time.sleep(0.5)
                except Exception:
                    break
        
        while True:
            try:
                close_popups()            
                show_more_button = WebDriverWait(driver, 2).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "a.button.btn-show-more.button__show-more-product"))
                )
                show_more_button.click()
                time.sleep(2)
            except Exception as e:
                break

        Page = driver.page_source
        Soup = BeautifulSoup(Page, "lxml")
        
        Product_Names = Soup.find_all("div", class_="product__name")
        Product_Prices = Soup.find_all("p", class_="product__price--show")

        Product_Information = {}

        if len(Product_Names) == len(Product_Prices):
            for name, price in zip(Product_Names, Product_Prices):
                product_name = name.get_text(strip=True)
                product_price = price.get_text(strip=True).replace("₫", "").replace(".", "").strip()
                Product_Information[product_name] = product_price
        else:
            print("Số lượng tên sản phẩm và giá sản phẩm không khớp!")

        for product, price in Product_Information.items():
            print(f"Tên sản phẩm: {product} - Giá: {price}")

    finally:
        driver.quit()
    
    df = pd.DataFrame(list(Product_Information.items()), columns=["Tên sản phẩm", "Giá"])
    df.to_csv(f"{Name}.csv", header=True, index=False)

In [5]:
def Scrape_Product_URL():
    URL = "https://cellphones.com.vn/"
    Homepage = requests.get(URL).text
    Soup = BeautifulSoup(Homepage, "lxml")

    Links = Soup.find_all("a", class_="multiple-link")

    Pages = {}
    for link in Links:
        Page_Name = link.get_text(strip=True).replace(",", "")
        Page_URL = link.get("href")
        Pages[Page_Name] = Page_URL

    return Pages

In [5]:
def Scrape_Per_URL():
    Links = Scrape_Product_URL()
    for Num in Links:
        Scrape_Product(Links[Num], Num)
        shutil.move(f"{Num}.csv", os.path.join("Cellphones Data","Dataset"))
Scrape_Per_URL()   

Tên sản phẩm: iPhone 16 128GB | Chính hãng VN/A - Giá: 22490000đ
Tên sản phẩm: iPhone 16 Pro Max 256GB | Chính hãng VN/A - Giá: 34990000đ
Tên sản phẩm: Samsung Galaxy S24 Ultra 12GB 256GB - Giá: 25990000đ
Tên sản phẩm: iPhone 15 Pro Max 256GB | Chính hãng VN/A - Giá: 29290000đ
Tên sản phẩm: Samsung Galaxy Z Fold6 - Giá: 38490000đ
Tên sản phẩm: iPhone 16 Pro 128GB | Chính hãng VN/A - Giá: 28990000đ
Tên sản phẩm: iPhone 13 128GB | Chính hãng VN/A - Giá: 14090000đ
Tên sản phẩm: Xiaomi 14T - Giá: 13990000đ
Tên sản phẩm: iPhone 15 128GB | Chính hãng VN/A - Giá: 19590000đ
Tên sản phẩm: iPhone 14 Pro Max 128GB | Chính hãng VN/A - Giá: 25590000đ
Tên sản phẩm: Samsung Galaxy A15 LTE 8GB 128GB - Giá: 4490000đ
Tên sản phẩm: Samsung Galaxy Z Flip6 - Giá: 24990000đ
Tên sản phẩm: Samsung Galaxy S24 FE 5G 8GB 128GB - Giá: 16990000đ
Tên sản phẩm: iPhone 15 Pro 128GB | Chính hãng VN/A - Giá: 25190000đ
Tên sản phẩm: iPhone 15 Plus 128GB | Chính hãng VN/A - Giá: 22590000đ
Tên sản phẩm: Samsung Galaxy S23

In [6]:
def Data_Cleaning():
    for File_Path in glob.glob(os.path.join("Cellphones Data", "Dataset", "*.csv")):
        File = pd.read_csv(File_Path)
        
        File["Giá"] = File["Giá"].str.replace("đ", "").str.strip()
        File["Trạng thái"] = File["Giá"].apply(lambda x: "Còn hàng" if x.isdigit() else "Hết hàng")
        File["Ngày"] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        File.to_csv(File_Path, index=False, encoding='utf-8-sig')

Data_Cleaning()

In [7]:
def Data_Cleaning():
    for File_Path in glob.glob(os.path.join("Cellphones Data", "Dataset", "*.csv")):
        File = pd.read_csv(File_Path)
        
        File["Giá"] = File["Giá"].str.replace("đ", "").str.strip()
        File["Trạng thái"] = File["Giá"].apply(lambda x: "Còn hàng" if x.isdigit() else "Hết hàng")
        File["Ngày"] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        File.to_csv(File_Path, index=False, encoding='utf-8-sig')       

Data_Cleaning()